In [2]:
# standard library imports
import json
import sys
# third party imports
from geopy.distance import distance
import numpy as np
from tqdm import tqdm
# local imports
sys.path.append('..')
from geo_llama.main import GeoLlama
from geo_llama.model import TopoModel, RAGModel
from geo_llama.translator import Translator


# Testing GeoLlama on foreign language text
In this notebook we will test the model on text from non-english sources. We have 15 texts in French, Traditional Chinese and Simplified Chinese.

We will test the model without the initial translation stage first, since Llama3 is desinged to have multi-lingual capability. The results will be compared against human annotated data.

In [ ]:
# set up the model
topo_model = TopoModel(model_name='JoeShingleton/GeoLlama_7b_toponym',
                       prompt_path='../data/prompt_templates/prompt_template.txt',
                       instruct_path='../data/prompt_templates/topo_instruction.txt',
                       input_path=None,
                       config_path='../data/config_files/model_config.json')

rag_model = RAGModel(model_name='JoeShingleton/GeoLlama_7b_RAG',
                       prompt_path='../data/prompt_templates/prompt_template.txt',
                       instruct_path='../data/prompt_templates/rag_instruction.txt',
                       input_path='../data/prompt_templates/rag_input.txt',
                       config_path='../data/config_files/model_config.json')

translator = Translator(model_size="1.2B")

geo_llama = GeoLlama(topo_model, rag_model, translate_model=translator)

In [9]:
results = []
for article in tqdm(range(1,16)):
  path = '../data/test_data/non_english_texts/article'+str(article)+'.txt'
  with open(path, 'r') as f:
    text = f.read()

  results.append(geo_llama.geoparse(text))

100%|██████████| 15/15 [06:06<00:00, 24.44s/it]


In [132]:
import pandas as pd
from geopy.distance import distance

true_data = pd.read_csv('../data/test_data/translated_toponyms.txt', encoding='utf-16', delimiter='\t')

correct_distance = []
precision = []
recall = []
for i, pred_toponyms in enumerate(results):
  true_toponyms = true_data[true_data.Article==i+1]
  correct_toponyms = []
  for pred in pred_toponyms:
    if pred['name'] in true_toponyms.Toponym.values:
      correct_toponyms.append(pred['name'])
      correct_location = (pred['latitude'], pred['longitude'])
      true_toponym  = true_toponyms[true_toponyms.Toponym==pred['name']]
      true_location = (true_toponym.Latitude.values[0], true_toponym.Longitude.values[0])
      correct_distance.append(distance(correct_location, true_location).km)
  TP = len(correct_toponyms)
  FP = len(pred_toponyms)-TP
  FN = len(true_toponyms)-TP
  if TP+FP==0:
    precision.append(0)
  else:
    precision.append(TP/(TP+FP))
  if TP+FN==0:
    recall.append(0)
  else:
    recall.append(TP/(TP+FN))


In [133]:
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall)
print(f'Macro precision: {macro_precision:.3f}')
print(f'Macro recall: {macro_recall:.3f}')
print(f'Macro F1: {macro_f1:.3f}')

Macro precision: 0.733
Macro recall: 0.794
Macro F1: 0.762


In [134]:
acc_at_1km = len([d for d in correct_distance if d<=1])/len(correct_distance)
acc_at_80km = len([d for d in correct_distance if d<=80])/len(correct_distance)
acc_at_161km = len([d for d in correct_distance if d<=161])/len(correct_distance)
mean_distance = np.mean(correct_distance)
median_distance = np.median(correct_distance)

print(f'Acc@1km: {acc_at_1km:.3f}')
print(f'Acc@80km: {acc_at_80km:.3f}')
print(f'Acc@161km: {acc_at_161km:.3f}')
print(f'Mean distance: {mean_distance:.3f}')
print(f'Median distance: {median_distance:.3f}')



Acc@1km: 0.640
Acc@80km: 0.880
Acc@161km: 0.960
Mean distance: 21.217
Median distance: 0.034


In [14]:
results = []
translated_texts = []
for article in tqdm(range(1,16)):
  path = '../data/test_data/non_english_texts/article'+str(article)+'.txt'
  with open(path, 'r') as f:
    text = f.read()

  translated_text = translator.translate(text)
  results.append(geo_llama.geoparse(translated_text['translation']))
  translated_texts.append(translated_text['translation'])

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translating 17 lines:



  7%|▋         | 1/15 [10:39<2:29:06, 639.07s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translating 7 lines:



 13%|█▎        | 2/15 [17:30<1:49:24, 504.95s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...


 87%|████████▋ | 13/15 [1:27:17<11:57, 358.95s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translating 18 lines:



 93%|█████████▎| 14/15 [1:37:14<07:11, 431.13s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Translating 7 lines:



100%|██████████| 15/15 [1:41:17<00:00, 405.17s/it]


In [20]:
with open('../data/results/translated_results.json', 'w') as f:
  json.dump(results, f)

In [17]:
import pandas as pd
from geopy.distance import distance

true_data = pd.read_csv('../data/test_data/translated_toponyms.txt', encoding='utf-16', delimiter='\t')

correct_distance = []
precision = []
recall = []
for i, pred_toponyms in enumerate(results):
  true_toponyms = true_data[true_data.Article==i+1]
  correct_toponyms = []
  for pred in pred_toponyms:
    if pred['name'] in true_toponyms.toponym_en.values:
      correct_toponyms.append(pred['name'])
      correct_location = (pred['latitude'], pred['longitude'])
      true_toponym  = true_toponyms[true_toponyms.toponym_en==pred['name']]
      true_location = (true_toponym.Latitude.values[0], true_toponym.Longitude.values[0])
      correct_distance.append(distance(correct_location, true_location).km)
  TP = len(correct_toponyms)
  FP = len(pred_toponyms)-TP
  FN = len(true_toponyms)-TP
  if TP+FP==0:
    precision.append(0)
  else:
    precision.append(TP/(TP+FP))
  if TP+FN==0:
    recall.append(0)
  else:
    recall.append(TP/(TP+FN))

In [18]:
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall)
print(f'Macro precision: {macro_precision:.3f}')
print(f'Macro recall: {macro_recall:.3f}')
print(f'Macro F1: {macro_f1:.3f}')

Macro precision: 0.513
Macro recall: 0.664
Macro F1: 0.579


In [19]:
acc_at_1km = len([d for d in correct_distance if d<=1])/len(correct_distance)
acc_at_80km = len([d for d in correct_distance if d<=80])/len(correct_distance)
acc_at_161km = len([d for d in correct_distance if d<=161])/len(correct_distance)
mean_distance = np.mean(correct_distance)
median_distance = np.median(correct_distance)

print(f'Acc@1km: {acc_at_1km:.3f}')
print(f'Acc@80km: {acc_at_80km:.3f}')
print(f'Acc@161km: {acc_at_161km:.3f}')
print(f'Mean distance: {mean_distance:.3f}')
print(f'Median distance: {median_distance:.3f}')

Acc@1km: 0.625
Acc@80km: 0.875
Acc@161km: 0.925
Mean distance: 270.736
Median distance: 0.034


## Assessing the translated toponyms
There are likely some instances where a translated toponym is correct, but the traslations used in the ground truth is different. For example the ground truth might have the lable "Unite States" and the model might predict "U.S.", or the ground truth might be "Hubei" and the label "Hubei Province". We do not want to unduly penalise the model in these instances, so we will manually assess the accuracy of the model against he ground truth. 


In [32]:
import pandas as pd
import json

with open('../data/results/translated_results.json', 'r') as f:
    pred_data = json.load(f)
    
true_data = pd.read_csv('../data/test_data/translated_toponyms.txt', encoding='utf-16', delimiter='\t')

In [79]:
precision = []
recall = []

In [80]:
idx = 0
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')


Predicted_toponyms ['Wuhan', 'China', 'Hubei']
True toponyms = ['Wuhan']


In [81]:
TP = len(['Whuan'])
FP = len(['China', 'Hubei'])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [82]:
idx = 1
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')


Predicted_toponyms ['China', 'U.S.', 'New York', 'United States']
True toponyms = ['United States' 'China' 'New York']


In [83]:
TP = len(['China', 'U.S.', 'New York'])
FP = len([])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [84]:
idx = 2
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Laos', 'China', 'Ukraine', 'Russia', 'U.S.', 'San Francisco', 'Russian', 'United States', 'Russian military', 'Laos']
True toponyms = ['United States' 'China' 'Vientiane' 'San Francisco' 'Laos' 'Russia'
 'Ukraine']


In [85]:
TP = len(['Laos', 'China', 'Ukraine', 'Russia', 'U.S.', 'San Francisco'])
FP = len(['Russian', 'Russian millitary'])
FN = len(['Vientiane'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [86]:
idx = 3
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['China', 'Hainan', 'Macao', 'Hong Kong', 'Hong Kong-Macao']
True toponyms = ['Macao' 'Hong Kong' 'Hainan']


In [87]:
TP = len(['China', 'Hainan', 'Macao', 'Hong Kong'])
FP = len(['Hong Kong-Macao'])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [88]:
idx = 4
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Senna River', 'Paris', 'France', 'Manna River', 'Seine']
True toponyms = ['Marne River' 'France' 'Paris' 'Seine River']


In [89]:
# As 'Senna River' and 'Manna River' are unlikely to be geolocated to the correct location, they will be considered incorrect
TP = len(['Paris', 'France', 'Seine'])
FP = len(['Senna River', 'Manna River'])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [90]:
idx = 5
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Hong Kong']
True toponyms = ['The Chinese University of Hong Kong']


In [91]:
# Hongkong is within the toponym 'Hong Kong University...' so is correct
TP = len([])
FP = len(['Hong Kong'])
FN = len(['The Chinese University of Hong Kong'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [92]:
idx = 6
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['China', 'Hong Kong', 'Fujian Province', 'China', 'Tianjin City', 'Xiamen', 'Tianjin']
True toponyms = ['Xiamen City' 'Fujian Province' 'China' 'Hong Kong' 'South China Sea'
 'Ludao' 'Wulao Peak']


In [93]:
TP = len(['China', 'Hong Kong', 'Fujian Province', 'Xiamen'])
FP = len(['Tianjin'])
FN = len(['South China Sea', 'Wulao Peak', 'Ludao'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [94]:
idx = 7
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Cuba', 'U.S.', 'Nicaragua', 'Panama', 'Venezuela', 'Uruguay', 'Argentina', 'Honduras', 'Bolivia']
True toponyms = ['Uruguay' 'Honduras' 'United States' 'Venezuela' 'China' 'Cuba'
 'Nicaragua' 'Bolivia' 'Argentina' 'Panama']


In [95]:
TP = len(['Cuba', 'U.S.', 'Nicaragua', 'Panama', 'Venezuela', 'Uruguay', 'Argentina', 'Honduras', 'Bolivia'])
FP = len([])
FN = len(['China'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [96]:
idx = 8
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Osaka', 'Japan', 'Osaka City', 'Guangzhou', 'Great Britain', 'Osaka Dreamland World', 'Guangxi', 'San Diego', 'Osaka Guangxi', 'Japan']
True toponyms = ['Kizugawa City' 'Japan' 'Stonehenge' 'United Kingdom' 'China' 'Kinki'
 'Osaka Castle' 'Kizugawa City' 'Kyoto']


In [97]:
TP = len(['Osaka', 'Japan', 'UK'])
FP = len(['Guanzhou', 'Osaka Dreamland Park', 'Guangxi', 'San Diego', 'Osaka Guangxi'])
FN = len(['Kizugawa city', 'Stonehenge', 'Kinki', 'Kyoto'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))


In [98]:
idx = 9
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Aikido County', '岐阜', 'Japan', 'County City', 'UNESCO', 'Tailand', 'UN']
True toponyms = ['Gushi City' 'Gifu Prefecture' 'Japan' 'Ichinomiya City'
 'Aichi Prefecture']


In [99]:
TP = len(['Japan'])
FP = len(['Aikido County', '岐阜', 'County City', 'UNESCO', 'Tailand', 'UN'])
FN = len(['Gushi City', 'Gifu Prefecture', 'Ichinomiya City', 'Aichi Prefecture'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [100]:
idx = 10
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['California']
True toponyms = ['California' 'San Francisco']


In [101]:
TP = len(['California'])
FP = len([])
FN = len(['San Francisco'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [102]:
idx = 11
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['France', 'Seine-Maritime', 'Rouen']
True toponyms = ['Seine-Maritime' 'Rouen']


In [103]:
TP = len(['Seine-Maritime', 'Rouen'])
FP = len(['France'])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [104]:
idx = 12
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Morocco', 'Tiflet', 'Rabat']
True toponyms = ['Morocco' 'Tiflet' 'Rabat']


In [105]:
TP = len(['Morocco', 'Tiflet', 'Rabat'])
FP = len([])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [106]:
idx = 13
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['U.S.', 'South China Sea', 'U.S. Secretary', 'Asia-Pacific', 'Moscow', 'Australia', 'China', 'Tokyo', 'Philippines', 'Japan', 'Indian Ocean', 'Manila', 'North Korea', 'Pacific', 'Beijing', 'Russia', 'Second Thomas', 'Palaos', 'Indo-Pacific', 'Indian', 'United States']
True toponyms = [nan 'United States' 'Japan' 'Australia' 'Beijing' 'China' 'North Korea'
 'Russia' 'Moscow' 'Ukraine' 'Palau' 'Manilla' 'India' 'Tokyo']


In [107]:
TP = len(['U.S.', 'Asia-Pacific', 'Moscow', 'Australia', 'China', 'Tokyo', 'Japan', 'Manila', 'North Korea', 'Beijing', 'Russia', 'Palaos', 'Indian'])
FP = len(['South China Sea', 'U.S. Secretary', 'Philippines', 'Indian Ocean', 'Pacific', 'Second Thomas', 'Indo-Pacific'])
FN = len([])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [108]:
idx = 14
true = true_data[true_data['Article']==idx+1]
pred = pred_data[idx]

print(f'Predicted_toponyms {[t['name'] for t in pred]}')
print(f'True toponyms = {true.toponym_en.values}')

Predicted_toponyms ['Paris', 'Beirut', 'Israel', 'Middle East', 'Lebanon']
True toponyms = ['Beirut' 'Lebanon' 'Charles-de-Gaulle']


In [109]:
TP = len(['Paris', 'Beirut', 'Lebanon'])
FP = len(['Israel', 'Middle East'])
FN = len(['Charles-de-Gaulle'])

precision.append(TP/(TP+FP))
recall.append(TP/(TP+FN))

In [110]:
import numpy as np
macro_precision = np.mean(precision)
macro_recall = np.mean(recall)
macro_f1 = 2*macro_precision*macro_recall/(macro_precision+macro_recall)
print(f'Macro precision: {macro_precision:.3f}')
print(f'Macro recall: {macro_recall:.3f}')
print(f'Macro F1: {macro_f1:.3f}')

Macro precision: 0.648
Macro recall: 0.747
Macro F1: 0.694
